In [1]:
import os
import numpy as np
from sklearn import metrics
from simpletransformers.classification import ClassificationModel
import pandas as pd
from scipy.special import softmax

## Utils

In [2]:
def extract_results_from_txt(contents):
    contents_list = list(filter(None,contents.split('\n')))
    results_dict = {}
    for element in contents_list:
        element_list = element.split("=")
        element_list = list(map(str.strip, element_list))
        results_dict[element_list[0]] = float(element_list[1])
    return results_dict

In [46]:
def build_results_dict(model_name, data_folder, preprocessing, new_data):
    general_results_dict = dict()
    for label in ["lost_job_1mo","is_unemployed", "job_search", "is_hired_1mo", "job_offer"]:
        general_results_dict[label] = dict()
        if model_name =="bert-base-cased-conversational" and preprocessing == '0' and new_data == '0':
            path_models = '/home/manuto/results_simpletransformers_{0}_{1}_{2}/DeepPavlov/{0}_{1}_{3}/models'.format(model_name,data_folder,str(preprocessing),label)
        elif model_name == "bert-base-uncased" and preprocessing == '1' and new_data == '0':
            path_models = '/home/manuto/results_simpletransformers_{0}_{1}_{2}/{0}_{1}_{3}_preprocessed/models'.format(model_name,data_folder,str(preprocessing),label)
        elif model_name == "bert-base-cased-conversational" and preprocessing == '0' and new_data == '1':
            path_models = '/home/manuto/results_simpletransformers_{0}_{1}_{2}_new_data_combined_split/DeepPavlov/{0}_{1}_{3}_new_data_combined_split/models'.format(model_name,data_folder,str(preprocessing),label)
        elif model_name == "bert-base-uncased" and preprocessing == '0' and new_data == '1':
            path_models = '/home/manuto/results_simpletransformers_{0}_{1}_{2}_new_data_combined_split/{0}_{1}_{3}_new_data_combined_split/models'.format(model_name,data_folder,str(preprocessing),label) 
        else:
            path_models = '/home/manuto/results_simpletransformers_bert-base-cased-conversational_jun3_10Klabels_nosampling_0/DeepPavlov/bert-base-cased-conversational_jun3_10Klabels_{}/models'.format(label)
            
        print(path_models)
        
        for epoch in os.listdir(path_models):
            if epoch == 'best_model': continue
            f=open(os.path.join(path_models, epoch, 'eval_results.txt'), "r")
            if f.mode =="r":
                contents=f.read()
            general_results_dict[label][epoch] = extract_results_from_txt(contents)
    return general_results_dict

In [47]:
def extract_best_epoch_list(epoch_list):
    epoch_number_list = []
    for epoch_name in epoch_list:
        if epoch_name != "checkpoint-2000" and epoch_name != "checkpoint-4000" and epoch_name != "checkpoint-6000" and epoch_name != "checkpoint-8000":
            epoch_number_list.append(float(epoch_name.split('-')[3]))
    index_best_epoch = np.argmin(epoch_number_list)
    return epoch_list[index_best_epoch]        

In [48]:
def keys_with_best_values(my_dict, factor):
    if factor == "f1":
        key_list = [key  for (key, value) in my_dict.items() if value == max(my_dict.values())]
        if len(key_list) == 1:
            return key_list[0]
        else:
            return extract_best_epoch_list(key_list)
    elif factor =="eval_loss":
        key_list = [key  for (key, value) in my_dict.items() if value == min(my_dict.values())]
        if len(key_list) == 1:
            return key_list[0]
        else:
            return extract_best_epoch_list(key_list)

In [54]:
def create_path_eval_data(data_folder, preprocessing, label, new_split):
    if data_folder == "may5_7Klabels" and preprocessing == '0':
        return '/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_{}.csv'.format(label)
    elif data_folder == "may5_7Klabels" and preprocessing =='1':
        return '/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/preprocessed_glove/val_{}.csv'.format(label)
    elif data_folder == "may20_9Klabels" and preprocessing == '0' and new_split == '0':
        return '/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_{}.csv'.format(label)  
    elif data_folder == "may20_9Klabels" and preprocessing == '0' and new_split == '1':
        return '/home/manuto/twitter/data/may28_iter1/val_{}.csv'.format(label)
    elif data_folder == "may20_9Klabels" and preprocessing == '1':
        return '/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/preprocessed_glove/val_{}.csv'.format(label)
    elif data_folder == "jun2" :
        return '/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_{}.csv'.format(label)
    elif data_folder == "jun3_10Klabels":
        return '/home/manuto/twitter/data/jun3_10Klabels/data_binary_pos_neg_unbalanced/val_{}.csv'.format(label)
    else:
        return 'not found'

In [64]:
def get_auc(model_type, model_folder_path, path_eval_data, preprocessing, new_split):
    model = ClassificationModel(model_type, model_folder_path, args={'evaluate_during_training': True, 'evaluate_during_training_verbose': True, 'num_train_epochs': 20})
    print(path_eval_data)
    eval_df = pd.read_csv(path_eval_data, lineterminator='\n')
    if preprocessing == "1" and new_split == '0':
        eval_df.columns = ["id", "text_raw", "labels", "text", "text_processed_length"]
    elif preprocessing == '0' and new_split == '0':
        eval_df.columns = ["id", "text", "labels"]
    else:
        eval_df.columns = ['text','labels']
    eval_df = eval_df[["text","labels"]]
    result, model_outputs, wrong_predictions = model.eval_model(eval_df)
    probabilities = np.array([softmax(element) for element in model_outputs])
    y_pred = np.array([element[1] for element in probabilities])
    eval_df['scores'] = y_pred
    fpr, tpr, thresholds = metrics.roc_curve(eval_df["labels"], y_pred)
    return [metrics.auc(fpr, tpr), eval_df]

In [82]:
def extract_best_epoch(factor, general_results_dict, model_type, model_name, data_folder, preprocessing, new_data,new_split, save=False):
    for label in general_results_dict.keys():
        results_per_label_dict = dict()
        for epoch in general_results_dict[label].keys():
            results_per_label_dict[epoch] = general_results_dict[label][epoch][factor]
        key_best_perf = keys_with_best_values(results_per_label_dict, factor)
        if model_name =="bert-base-cased-conversational" and preprocessing == '0' and new_data == '0':
            path_models = '/home/manuto/results_simpletransformers_{0}_{1}_{2}/DeepPavlov/{0}_{1}_{3}/models'.format(model_name,data_folder,str(preprocessing),label)
        elif model_name == "bert-base-uncased" and preprocessing == '1' and new_data == '0':
            path_models = '/home/manuto/results_simpletransformers_{0}_{1}_{2}/{0}_{1}_{3}_preprocessed/models'.format(model_name,data_folder,str(preprocessing),label)
        elif model_name == "bert-base-cased-conversational" and preprocessing == '0' and new_data == '1':
            path_models = '/home/manuto/results_simpletransformers_{0}_{1}_{2}_new_data_combined_split/DeepPavlov/{0}_{1}_{3}_new_data_combined_split/models'.format(model_name,data_folder,str(preprocessing),label)
        elif model_name == "bert-base-uncased" and preprocessing == '0' and new_data == '1':
            path_models = '/home/manuto/results_simpletransformers_{0}_{1}_{2}_new_data_combined_split/{0}_{1}_{3}_new_data_combined_split/models'.format(model_name,data_folder,str(preprocessing),label) 
        else:
            path_models = '/home/manuto/results_simpletransformers_{0}_{1}_nosampling_{2}/DeepPavlov/{0}_jun3_10Klabels_{3}/models'.format(model_name,data_folder,str(preprocessing),label)
        path_best_epoch = os.path.join(path_models,key_best_perf)
        path_eval_data = create_path_eval_data(data_folder, preprocessing, label, new_split)
        print('***************',label,'***************')
        print('Best epoch', key_best_perf)
        print('Precision', general_results_dict[label][key_best_perf]['precision'])
        print('Recall', general_results_dict[label][key_best_perf]['recall'])
        print('F1 score', general_results_dict[label][key_best_perf]['f1'])
        print('eval_loss', general_results_dict[label][key_best_perf]['eval_loss'])
        AUC, eval_df = get_auc(model_type, path_best_epoch, path_eval_data, preprocessing, new_split)
        print('AUC', AUC)
        if save:
            if not os.path.exists(os.path.join(os.path.dirname(path_eval_data), '{}_scores'.format(model_name))):
                os.makedirs(os.path.join(os.path.dirname(path_eval_data), '{}_scores'.format(model_name)))
            path_eval_df_scores = os.path.join(os.path.dirname(path_eval_data), '{}_scores'.format(model_name),'val_{}.csv'.format(label))
            eval_df.to_csv(path_eval_df_scores, index=False)
            print("Eval dataframe with scores was saved at: ", path_eval_df_scores)

## BERT base uncased on may5_7Klabels, no preprocessing

In [61]:
general_results_dict = build_results_dict('bert-base-uncased', 'may5_7Klabels', '0')

TypeError: build_results_dict() missing 1 required positional argument: 'new_data'

In [45]:
extract_best_epoch('f1', general_results_dict, 'bert', 'bert-base-uncased','may5_7Klabels',0)

*************** lost_job_1mo ***************
Best epoch checkpoint-816-epoch-6
Precision 0.8823529411764706
Recall 0.9375
F1 score 0.9090909090909091
eval_loss 0.6023551852387541
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=272.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=34.0), HTML(value='')))


AUC 0.9500868055555556
*************** is_unemployed ***************
Best epoch checkpoint-1614-epoch-6
Precision 0.8790322580645161
Recall 0.8416988416988417
F1 score 0.8599605522682445
eval_loss 0.9401171807922534
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=536.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=28.0), HTML(value='')))


AUC 0.9604435617345249
*************** job_offer ***************
Best epoch checkpoint-4437-epoch-17
Precision 0.9634146341463414
Recall 0.948
F1 score 0.9556451612903225
eval_loss 0.39379104068785004
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=521.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=66.0), HTML(value='')))


AUC 0.9863911439114391


In [47]:
extract_best_epoch('eval_loss', general_results_dict, 'bert', 'bert-base-uncased','may5_7Klabels',0)

*************** lost_job_1mo ***************
Best epoch checkpoint-272-epoch-2
Precision 0.8807947019867549
Recall 0.9236111111111112
F1 score 0.9016949152542373
eval_loss 0.2525600970667951
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=272.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=34.0), HTML(value='')))


AUC 0.9615342881944444
*************** is_unemployed ***************
Best epoch checkpoint-269-epoch-1
Precision 0.8006993006993007
Recall 0.8841698841698842
F1 score 0.8403669724770643
eval_loss 0.3835225437781704
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=536.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=28.0), HTML(value='')))


AUC 0.9401688182720953
*************** job_offer ***************
Best epoch checkpoint-261-epoch-1
Precision 0.9236641221374046
Recall 0.968
F1 score 0.9453124999999999
eval_loss 0.18365865753907146
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=521.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=66.0), HTML(value='')))


AUC 0.982


## BERT base uncased on may5_7Klabels, preprocessing

In [12]:
general_results_dict = build_results_dict('bert-base-uncased', 'may5_7Klabels', '1')

In [13]:
extract_best_epoch('f1', general_results_dict, 'bert', 'bert-base-uncased','may5_7Klabels','1')

*************** lost_job_1mo ***************
Best epoch checkpoint-952-epoch-7
Precision 0.9256756756756757
Recall 0.9513888888888888
F1 score 0.9383561643835616
eval_loss 0.4923343294683625
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/preprocessed_glove/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=272.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=34.0), HTML(value='')))


AUC 0.9728461371527778
*************** is_unemployed ***************
Best epoch checkpoint-4304-epoch-16
Precision 0.8893129770992366
Recall 0.8996138996138996
F1 score 0.8944337811900192
eval_loss 1.002681013585916
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/preprocessed_glove/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=536.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=28.0), HTML(value='')))


AUC 0.9757116848725587
*************** job_offer ***************
Best epoch checkpoint-1566-epoch-6
Precision 0.9877551020408163
Recall 0.968
F1 score 0.9777777777777779
eval_loss 0.13232990218834442
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/preprocessed_glove/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=521.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=66.0), HTML(value='')))


AUC 0.9943173431734318


In [14]:
extract_best_epoch('eval_loss', general_results_dict, 'bert', 'bert-base-uncased','may5_7Klabels','1')

*************** lost_job_1mo ***************
Best epoch checkpoint-272-epoch-2
Precision 0.8670886075949367
Recall 0.9513888888888888
F1 score 0.9072847682119205
eval_loss 0.27636635215843425
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/preprocessed_glove/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=272.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=34.0), HTML(value='')))


AUC 0.9619140625
*************** is_unemployed ***************
Best epoch checkpoint-269-epoch-1
Precision 0.8369565217391305
Recall 0.8918918918918919
F1 score 0.8635514018691588
eval_loss 0.3482618156208921
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/preprocessed_glove/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=536.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=28.0), HTML(value='')))


AUC 0.980883813306852
*************** job_offer ***************
Best epoch checkpoint-1566-epoch-6
Precision 0.9877551020408163
Recall 0.968
F1 score 0.9777777777777779
eval_loss 0.13232990218834442
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/preprocessed_glove/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=521.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=66.0), HTML(value='')))


AUC 0.9943173431734318


## BERT base uncased on may20_9Klabels, no preprocessing

In [9]:
general_results_dict = build_results_dict('bert-base-uncased', 'may20_9Klabels', '0')

In [10]:
extract_best_epoch('f1', general_results_dict ,'bert', 'bert-base-uncased','may20_9Klabels',0)

*************** lost_job_1mo ***************
Best epoch checkpoint-2160-epoch-12
Precision 0.8638743455497382
Recall 0.9116022099447514
F1 score 0.8870967741935484
eval_loss 1.0054219997591443
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=358.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=45.0), HTML(value='')))


AUC 0.9219340137965478
*************** is_unemployed ***************
Best epoch checkpoint-732-epoch-2
Precision 0.7955112219451371
Recall 0.8598382749326146
F1 score 0.8264248704663213
eval_loss 0.41010259091854095
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=732.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=38.0), HTML(value='')))


AUC 0.9393457519309404
*************** job_offer ***************
Best epoch checkpoint-2000
Precision 0.9385474860335196
Recall 0.9411764705882353
F1 score 0.9398601398601398
eval_loss 0.33165798987122785
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=692.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=87.0), HTML(value='')))


AUC 0.9800995024875622


In [11]:
extract_best_epoch('eval_loss', general_results_dict ,'bert', 'bert-base-uncased','may20_9Klabels',0)

*************** lost_job_1mo ***************
Best epoch checkpoint-360-epoch-2
Precision 0.8133971291866029
Recall 0.9392265193370166
F1 score 0.8717948717948718
eval_loss 0.3715955116682582
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=358.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=45.0), HTML(value='')))


AUC 0.9271467365858226
*************** is_unemployed ***************
Best epoch checkpoint-732-epoch-2
Precision 0.7955112219451371
Recall 0.8598382749326146
F1 score 0.8264248704663213
eval_loss 0.41010259091854095
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=732.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=38.0), HTML(value='')))


AUC 0.9304407087687414
*************** job_offer ***************
Best epoch checkpoint-347-epoch-1
Precision 0.8967391304347826
Recall 0.9243697478991597
F1 score 0.910344827586207
eval_loss 0.26268179920212975
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=692.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=87.0), HTML(value='')))


AUC 0.9682010117479828


## BERT base uncased on may20_9Klabels combined with new labels, eval on may20 val data, no preprocessing

In [18]:
general_results_dict = build_results_dict('bert-base-uncased', 'may20_9Klabels', '0', '1')

In [19]:
extract_best_epoch('f1', general_results_dict ,'bert', 'bert-base-uncased','may20_9Klabels','0', '1', '0')

*************** lost_job_1mo ***************
Best epoch checkpoint-4768-epoch-16
Precision 0.8702702702702703
Recall 0.8895027624309392
F1 score 0.8797814207650274
eval_loss 1.2038969511787097
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=358.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=45.0), HTML(value='')))


AUC 0.9114461404001624
*************** is_unemployed ***************
Best epoch checkpoint-5808-epoch-12
Precision 0.8554913294797688
Recall 0.7978436657681941
F1 score 0.8256624825662483
eval_loss 1.3882976793886528
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=732.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=38.0), HTML(value='')))


AUC 0.935029532030895
*************** job_offer ***************
Best epoch checkpoint-6990-epoch-15
Precision 0.9299719887955182
Recall 0.9299719887955182
F1 score 0.9299719887955182
eval_loss 0.5195747796317627
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=692.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=87.0), HTML(value='')))


AUC 0.9673648563903173


In [20]:
extract_best_epoch('eval_loss', general_results_dict ,'bert', 'bert-base-uncased','may20_9Klabels','0', '1', '0')

*************** lost_job_1mo ***************
Best epoch checkpoint-596-epoch-2
Precision 0.850828729281768
Recall 0.850828729281768
F1 score 0.850828729281768
eval_loss 0.461541814605395
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=358.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=45.0), HTML(value='')))


AUC 0.9169397883696975
*************** is_unemployed ***************
Best epoch checkpoint-968-epoch-2
Precision 0.8634920634920635
Recall 0.7331536388140162
F1 score 0.793002915451895
eval_loss 0.4684803126298863
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=732.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=38.0), HTML(value='')))


AUC 0.935029532030895
*************** job_offer ***************
Best epoch checkpoint-932-epoch-2
Precision 0.9128065395095368
Recall 0.938375350140056
F1 score 0.925414364640884
eval_loss 0.231053262263879
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=692.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=87.0), HTML(value='')))


AUC 0.9745223462519336


## BERT base uncased on may20_9Klabels combined with new labels + new split, no preprocessing

In [65]:
general_results_dict = build_results_dict('bert-base-uncased', 'may20_9Klabels', '0', '1')

In [66]:
extract_best_epoch('f1', general_results_dict ,'bert', 'bert-base-uncased','may20_9Klabels','0', '1')

*************** lost_job_1mo ***************
Best epoch checkpoint-480-epoch-2
Precision 0.7855072463768116
Recall 0.8973509933774835
F1 score 0.8377125193199382
eval_loss 0.42481086526102235
/home/manuto/twitter/data/may28_iter1/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=823.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=103.0), HTML(value='')))


AUC 0.9266820048048201
*************** is_unemployed ***************
Best epoch checkpoint-2821-epoch-7
Precision 0.7745098039215687
Recall 0.7808896210873146
F1 score 0.7776866283839213
eval_loss 1.3314229151330932
/home/manuto/twitter/data/may28_iter1/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=1380.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=92.0), HTML(value='')))


AUC 0.9625578703703703
*************** job_offer ***************
Best epoch checkpoint-3096-epoch-8
Precision 0.91651865008881
Recall 0.926391382405745
F1 score 0.9214285714285715
eval_loss 0.38584687435124293
/home/manuto/twitter/data/may28_iter1/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=1325.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=166.0), HTML(value='')))


AUC 0.9636117968282465


In [67]:
extract_best_epoch('eval_loss', general_results_dict ,'bert', 'bert-base-uncased','may20_9Klabels','0', '1')

*************** lost_job_1mo ***************
Best epoch checkpoint-480-epoch-2
Precision 0.7855072463768116
Recall 0.8973509933774835
F1 score 0.8377125193199382
eval_loss 0.42481086526102235
/home/manuto/twitter/data/may28_iter1/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=823.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=103.0), HTML(value='')))


AUC 0.9266820048048201
*************** is_unemployed ***************
Best epoch checkpoint-403-epoch-1
Precision 0.7267355982274741
Recall 0.8105436573311368
F1 score 0.7663551401869159
eval_loss 0.4496568037853765
/home/manuto/twitter/data/may28_iter1/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=1380.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=92.0), HTML(value='')))


AUC 0.9426917989417989
*************** job_offer ***************
Best epoch checkpoint-774-epoch-2
Precision 0.8686371100164204
Recall 0.9497307001795332
F1 score 0.9073756432246998
eval_loss 0.2967144891350384
/home/manuto/twitter/data/may28_iter1/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=1325.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=166.0), HTML(value='')))


AUC 0.9559816352483543


## BERT base cased conversational on may5_7Klabels, no preprocessing

In [49]:
general_results_dict = build_results_dict('bert-base-cased-conversational', 'may5_7Klabels', '0')

In [50]:
extract_best_epoch('f1', general_results_dict ,'bert', 'bert-base-cased-conversational','may5_7Klabels',0)

*************** lost_job_1mo ***************
Best epoch checkpoint-2040-epoch-15
Precision 0.9121621621621622
Recall 0.9375
F1 score 0.9246575342465754
eval_loss 0.6794325965730583
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=272.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=34.0), HTML(value='')))


AUC 0.9675021701388888
*************** is_unemployed ***************
Best epoch checkpoint-2000
Precision 0.9176954732510288
Recall 0.861003861003861
F1 score 0.8884462151394422
eval_loss 0.7357624607966907
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=536.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=28.0), HTML(value='')))


AUC 0.9897384971863622
*************** job_offer ***************
Best epoch checkpoint-3393-epoch-13
Precision 0.98
Recall 0.98
F1 score 0.98
eval_loss 0.14672867931199796
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=521.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=66.0), HTML(value='')))


AUC 0.9967527675276753


In [51]:
extract_best_epoch('eval_loss', general_results_dict, 'bert', 'bert-base-cased-conversational','may5_7Klabels',0)

*************** lost_job_1mo ***************
Best epoch checkpoint-136-epoch-1
Precision 0.9
Recall 0.9375
F1 score 0.9183673469387755
eval_loss 0.2329696272225941
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=272.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=34.0), HTML(value='')))


AUC 0.9629991319444444
*************** is_unemployed ***************
Best epoch checkpoint-269-epoch-1
Precision 0.8671586715867159
Recall 0.9073359073359073
F1 score 0.8867924528301887
eval_loss 0.2612938166443092
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=536.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=28.0), HTML(value='')))


AUC 0.9834491890102616
*************** job_offer ***************
Best epoch checkpoint-1305-epoch-5
Precision 0.9918032786885246
Recall 0.968
F1 score 0.9797570850202428
eval_loss 0.10163646575176355
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=521.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=66.0), HTML(value='')))


AUC 0.9965018450184503


## BERT base cased conversational on May20_9Klabels, no preprocessing

In [20]:
general_results_dict = build_results_dict('bert-base-cased-conversational', 'may20_9Klabels', '0')

In [21]:
extract_best_epoch('f1', general_results_dict ,'bert', 'bert-base-cased-conversational','may20_9Klabels','0')

*************** lost_job_1mo ***************
Best epoch checkpoint-2520-epoch-14
Precision 0.8924731182795699
Recall 0.9171270718232044
F1 score 0.9046321525885558
eval_loss 0.8881418499681685
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=358.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=45.0), HTML(value='')))


AUC 0.9275213034928363
*************** is_unemployed ***************
Best epoch checkpoint-1464-epoch-4
Precision 0.899135446685879
Recall 0.8409703504043127
F1 score 0.8690807799442898
eval_loss 0.6348353032346653
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=732.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=38.0), HTML(value='')))


AUC 0.9542026351658337
*************** job_offer ***************
Best epoch checkpoint-5205-epoch-15
Precision 0.9429347826086957
Recall 0.9719887955182073
F1 score 0.9572413793103448
eval_loss 0.4400819239431414
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=692.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=87.0), HTML(value='')))


AUC 0.9837618629541369


In [23]:
extract_best_epoch('eval_loss', general_results_dict ,'bert', 'bert-base-cased-conversational','may20_9Klabels','0')

*************** lost_job_1mo ***************
Best epoch checkpoint-360-epoch-2
Precision 0.8652849740932642
Recall 0.9226519337016574
F1 score 0.893048128342246
eval_loss 0.31201827973127366
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=358.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=45.0), HTML(value='')))


AUC 0.9433155414052502
*************** is_unemployed ***************
Best epoch checkpoint-366-epoch-1
Precision 0.8337531486146096
Recall 0.8921832884097035
F1 score 0.8619791666666667
eval_loss 0.3495856652279263
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=732.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=38.0), HTML(value='')))


AUC 0.9720127214902317
*************** job_offer ***************
Best epoch checkpoint-1041-epoch-3
Precision 0.9578651685393258
Recall 0.9551820728291317
F1 score 0.9565217391304349
eval_loss 0.2027366018757738
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=692.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=87.0), HTML(value='')))


AUC 0.9896734813328317


## ALBERT base v2 on may5_7Klabels, no preprocessing

In [11]:
general_results_dict = build_results_dict('albert-base-v2', 'may5_7Klabels', '0')

In [12]:
extract_best_epoch('f1', general_results_dict, 'albert', 'albert-base-v2','may5_7Klabels',0)

*************** lost_job_1mo ***************
Best epoch checkpoint-1632-epoch-12
Precision 0.8741721854304636
Recall 0.9166666666666666
F1 score 0.8949152542372881
eval_loss 0.6131892136352903
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=272.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=34.0), HTML(value='')))


AUC 0.9245876736111112
*************** is_unemployed ***************
Best epoch checkpoint-4842-epoch-18
Precision 0.8699186991869918
Recall 0.8262548262548263
F1 score 0.8475247524752476
eval_loss 0.7184976005732123
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=536.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=28.0), HTML(value='')))


AUC 0.9663190996358821
*************** job_offer ***************
Best epoch checkpoint-2349-epoch-9
Precision 0.9833333333333333
Recall 0.944
F1 score 0.9632653061224489
eval_loss 0.2319823851188024
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=521.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=66.0), HTML(value='')))


AUC 0.987690036900369


In [13]:
extract_best_epoch('eval_loss', general_results_dict, 'albert', 'albert-base-v2','may5_7Klabels',0)

*************** lost_job_1mo ***************
Best epoch checkpoint-272-epoch-2
Precision 0.8424657534246576
Recall 0.8541666666666666
F1 score 0.8482758620689655
eval_loss 0.3698170752648045
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=272.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=34.0), HTML(value='')))


AUC 0.9328342013888888
*************** is_unemployed ***************
Best epoch checkpoint-2152-epoch-8
Precision 0.8421052631578947
Recall 0.803088803088803
F1 score 0.8221343873517786
eval_loss 0.4525177482126364
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=536.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=28.0), HTML(value='')))


AUC 0.9184045018205892
*************** job_offer ***************
Best epoch checkpoint-522-epoch-2
Precision 0.9251968503937008
Recall 0.94
F1 score 0.9325396825396824
eval_loss 0.20270754611401848
/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=521.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=66.0), HTML(value='')))


AUC 0.9820516605166052


## ALBERT base v2 on may20_9Klabels, no preprocessing

In [12]:
general_results_dict = build_results_dict('albert-base-v2', 'may20_9Klabels', '0')

In [14]:
extract_best_epoch('f1', general_results_dict, 'albert', 'albert-base-v2','may20_9Klabels',0)

*************** lost_job_1mo ***************
Best epoch checkpoint-2340-epoch-13
Precision 0.873015873015873
Recall 0.9116022099447514
F1 score 0.8918918918918919
eval_loss 0.6513210713863373
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=358.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=45.0), HTML(value='')))


AUC 0.9001623123263727
*************** is_unemployed ***************
Best epoch checkpoint-6954-epoch-19
Precision 0.8351648351648352
Recall 0.8194070080862533
F1 score 0.8272108843537415
eval_loss 0.8933284085081972
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=732.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=38.0), HTML(value='')))


AUC 0.9060427078600635
*************** job_offer ***************
Best epoch checkpoint-4000
Precision 0.9245810055865922
Recall 0.927170868347339
F1 score 0.9258741258741259
eval_loss 0.4513422276439338
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=692.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=87.0), HTML(value='')))


AUC 0.9651114177014088


In [15]:
extract_best_epoch('eval_loss', general_results_dict, 'albert', 'albert-base-v2','may20_9Klabels',0)

*************** lost_job_1mo ***************
Best epoch checkpoint-540-epoch-3
Precision 0.8342245989304813
Recall 0.861878453038674
F1 score 0.8478260869565217
eval_loss 0.39755570938189827
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=358.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=45.0), HTML(value='')))


AUC 0.9113837125823268
*************** is_unemployed ***************
Best epoch checkpoint-2562-epoch-7
Precision 0.8352941176470589
Recall 0.7654986522911051
F1 score 0.7988748241912799
eval_loss 0.472988398745656
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=732.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=38.0), HTML(value='')))


AUC 0.8813721035892775
*************** job_offer ***************
Best epoch checkpoint-1041-epoch-3
Precision 0.8970189701897019
Recall 0.927170868347339
F1 score 0.9118457300275483
eval_loss 0.2471186801277358
/home/manuto/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=692.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=87.0), HTML(value='')))


AUC 0.9645888206028681


## BERT base cased conversational on jun3_10Klabels data: all labels without sampling

In [80]:
general_results_dict = build_results_dict('convBERT', 'jun3_10Klabels', '0', '0')

/home/manuto/results_simpletransformers_bert-base-cased-conversational_jun3_10Klabels_nosampling_0/DeepPavlov/bert-base-cased-conversational_jun3_10Klabels_lost_job_1mo/models
/home/manuto/results_simpletransformers_bert-base-cased-conversational_jun3_10Klabels_nosampling_0/DeepPavlov/bert-base-cased-conversational_jun3_10Klabels_is_unemployed/models
/home/manuto/results_simpletransformers_bert-base-cased-conversational_jun3_10Klabels_nosampling_0/DeepPavlov/bert-base-cased-conversational_jun3_10Klabels_job_search/models
/home/manuto/results_simpletransformers_bert-base-cased-conversational_jun3_10Klabels_nosampling_0/DeepPavlov/bert-base-cased-conversational_jun3_10Klabels_is_hired_1mo/models
/home/manuto/results_simpletransformers_bert-base-cased-conversational_jun3_10Klabels_nosampling_0/DeepPavlov/bert-base-cased-conversational_jun3_10Klabels_job_offer/models


In [70]:
extract_best_epoch('f1', general_results_dict, 'bert', 'bert-base-cased-conversational','jun3_10Klabels','0','2','2', save=False)

*************** lost_job_1mo ***************
Best epoch checkpoint-5016-epoch-6
Precision 0.7725490196078432
Recall 0.7111913357400722
F1 score 0.7406015037593985
eval_loss 0.2930764340946461
/home/manuto/twitter/data/jun3_10Klabels/data_binary_pos_neg_unbalanced/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=2864.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=358.0), HTML(value='')))


AUC 0.9244284460346721
*************** is_unemployed ***************
Best epoch checkpoint-6000
Precision 0.7966101694915254
Recall 0.7594254937163375
F1 score 0.7775735294117647
eval_loss 0.5100327761396349
/home/manuto/twitter/data/jun3_10Klabels/data_binary_pos_neg_unbalanced/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=2864.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=358.0), HTML(value='')))


AUC 0.9698887002066412
*************** job_offer ***************
Best epoch checkpoint-4000
Precision 0.8506711409395973
Recall 0.9234972677595629
F1 score 0.8855895196506549
eval_loss 0.2682174239619817
/home/manuto/twitter/data/jun3_10Klabels/data_binary_pos_neg_unbalanced/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=2864.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=358.0), HTML(value='')))


AUC 0.9721079362831302


In [83]:
extract_best_epoch('eval_loss', general_results_dict, 'bert', 'bert-base-cased-conversational','jun3_10Klabels','0','2','2', save=True)

*************** lost_job_1mo ***************
Best epoch checkpoint-836-epoch-1
Precision 0.7111913357400722
Recall 0.7111913357400722
F1 score 0.7111913357400723
eval_loss 0.17294659488444222
/home/manuto/twitter/data/jun3_10Klabels/data_binary_pos_neg_unbalanced/val_lost_job_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=2864.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=358.0), HTML(value='')))


AUC 0.9461944546392056
Eval dataframe with scores was saved at:  /home/manuto/twitter/data/jun3_10Klabels/data_binary_pos_neg_unbalanced/bert-base-cased-conversational_scores/val_lost_job_1mo.csv
*************** is_unemployed ***************
Best epoch checkpoint-1672-epoch-2
Precision 0.7696526508226691
Recall 0.755834829443447
F1 score 0.7626811594202898
eval_loss 0.27779876143156484
/home/manuto/twitter/data/jun3_10Klabels/data_binary_pos_neg_unbalanced/val_is_unemployed.csv


HBox(children=(FloatProgress(value=0.0, max=2864.0), HTML(value='')))

...

HBox(children=(FloatProgress(value=0.0, max=358.0), HTML(value='')))


AUC 0.9698887002066412
Eval dataframe with scores was saved at:  /home/manuto/twitter/data/jun3_10Klabels/data_binary_pos_neg_unbalanced/bert-base-cased-conversational_scores/val_is_hired_1mo.csv
*************** job_offer ***************
Best epoch checkpoint-1672-epoch-2
Precision 0.8929292929292929
Recall 0.8051001821493625
F1 score 0.846743295019157
eval_loss 0.1866596001070305
/home/manuto/twitter/data/jun3_10Klabels/data_binary_pos_neg_unbalanced/val_job_offer.csv


HBox(children=(FloatProgress(value=0.0, max=2864.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=358.0), HTML(value='')))


AUC 0.9712762651119059
Eval dataframe with scores was saved at:  /home/manuto/twitter/data/jun3_10Klabels/data_binary_pos_neg_unbalanced/bert-base-cased-conversational_scores/val_job_offer.csv
